# The `Titanic` dataset

Go to the following `kaggle` webpage [https://www.kaggle.com/c/titanic/data](https://www.kaggle.com/c/titanic/data)

If you don't have a `kaggle` account, create one.

Download the `train.csv` dataset.

Answer the following questions using the libraries `pandas`, `seaborn` and/or any other library that you like.
We expect you to produce **nice looking plots**, at least **readable** and perform the required computations
 as concisely as possible using the `pandas` API. This is the way we will evaluate this homework.

Before boarding on the homework, import (install them if needed)
the next modules.




In [ ]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pdpipe as pdp

import requests

import regex
import re
import os
from collections import namedtuple
from collections import abc
from statsmodels.graphics.mosaicplot import mosaic

sns.set_context("notebook", font_scale=1.2)

The first thing to do is to read the description of
the columns and understand what the dataset is about and what it contains.

## Question : inspecting a DataFrame

Open the dataframe with `pandas`, check the first lines of the dataset

### Answer

In [ ]:
train = pd.read_csv("train.csv")
train.head()


Method `info`  print a a concise summary of a `DataFrame`.
With optional parameter `memory_usage`, you can get an estimate
of the amount of memory used by the `DataFrame`. Beware that
the resulting estimate depends on the argument fed.



In [ ]:
train.info(memory_usage=True)


## Question: Checking the types of columns

Check the types of the columns.

- How do you obtain the type of a column?
- How do you obtain the datatype (`dtype`) of elements of a column?

### Answer



In [ ]:
train.dtypes


## Question: selecting columns according to datatype

Project (in the SQL sense)  the DataFrame on columns with a specified datatype.
For example, project (as in the SQL `SELECT`)
the Titanic `DataFrame` on the  numerical columns.

Hint: method `select_dtypes`

### Answer


In [ ]:
train.select_dtypes(np.number)


## Question: computing number of distinct values in each column

For each column with non-numerical datatype compute the number of unique values
in the column.

### Answer



In [ ]:
nn = train.select_dtypes(exclude=np.number)
for i in nn:
    print(i + " " + str(len(train[i].unique())))


## Question: spotting categorical columns

What are the indices and column names of columns
with less than `10` distinct vakues ?

### Answer



In [ ]:
for col in train.columns:
    if len(train[col].unique()) <= 10:
        print(col, train.columns.get_loc(col))


## Question: coercing column datatypes

- `Name` should be of type `string`
- `Sex` is a categorical variable with values `Female, Male, None`
- `Embarked` is a categorical variable with values `Cherbourg, Southampton, ..., None`
- `Ticket` should be a string
- `Cabin` should be a string
- `Survived` should be of type `bool`
- `Pclass` is a categorical variable (ordered)
- `Sibsp` could a categorical variable (ordered)
- `Parch` could a categorical variable (ordered)

Attempt to automatically infer better types fails.




### Answer



In [ ]:
train['Survived'] = train['Survived'].astype(bool)
#train[['Name','Ticket','Cabin']] = train[['Name','Ticket','Cabin']].astype('str')

train[['Sex', 'Embarked']] = train[['Sex', 'Embarked']].apply(lambda x: x.astype('category'))

for i in ['Pclass','SibSp','Parch']:
    train[i] = pd.Categorical(train[i], ordered = True)


print(train.dtypes)

print(train['SibSp'].cat.categories)

train['SibSp'].cat.ordered


## Question: coercing column datatypes

All suspicious columns could be converted to `category`.
Some of the categories have to be ordered.


### Answer



## Question: categorizing columns using functions

Define a function that takes as arguments a `Dataframe` and a list of
of column names and attempts to cast the corresponding columns to 'category'.

### Answer




In [ ]:
def categoriseur(df, names):
    for i in (names):
        df[i] = df[i].astype('category')

#categoriseur(train,['Sex'])      



## Question: enforcing datatypes at loading time


Function `read_csv` from `Pandas` has many optional keyword arguments that
can be used to tweak the resulting `Dataframe` during (or just after load).

Improve your import of the data: for instance,
categorical columns (a column with a small number of modalities)
should have the `category` type.

### Answer


In [ ]:
new_dtypes = {'Sex': 'category', 'Embarked': 'category','Pclass': 'category',
              'SibSp': 'category','Parch': 'category'}
    
df = pd.read_csv("train.csv", dtype=new_dtypes)
df.dtypes

## Question: Count the number of missing values in each column

Display the distribution of `Pclass`  and `Cabin == NaN` in
the subpopulation defined by the missingness of `Age`.

Hint: you may use function `mosaic` from package `statsmodels`. See 
`from statsmodels.graphics.mosaicplot import mosaic`

### Answer

In [ ]:

mosaic(train[train['Age'].isna()].assign(Cabin_NaN=train['Cabin'].isna()).sort_values(by='Pclass'), ['Pclass', 'Cabin_NaN'], gap=0.01)

plt.title(' Distribution of Pclass and Cabin and Age are missing ')
plt.show() 



## Question: number of missing values per row

It is also possible to determine the number of NA values per row.
List the rows with at least two missing values. Can you complete
some dots?

### Answer

In [ ]:
ln = train.isna().sum(axis=1)

ln = ln[ln>=2].index.values

na_2r = train.iloc[ln]

#Proportion of passengers with 2 NA or more per class
print(na_2r['Pclass'].value_counts()/train['Pclass'].value_counts())


#the 3rd class passengers are overrepresented




## Question: describe, showing quintiles

For each numerical column, show the *quintiles* (from $20\%$ up to $80\%$).

### Answer



In [ ]:
train.quantile([0.2,0.8], numeric_only=True)



## Question: binning the `Age` column

Create a new categorical column named `Status`.

The value should be should depend on the `Age`
column with bins  `[0,17], (17, 40], (40, 60], (60, 100]`
(`np.nan` for passengers whose age is unknown).

### Answer




In [ ]:
Status = ''

b3 = train['Status']  = np.where(train['Age']<=100,"(60,100]", np.nan)
b2 = train['Status']  = np.where(train['Age']<=60,"(40,60]", b3)
b1 = train['Status']  = np.where(train['Age']<=40,"(17,40]", b2)
train['Status']  = np.where(train['Age']<=17,"(0,17]",b1)


categoriseur(train,['Status']) 

train.head(n=10)



## Question: 

Draw a `countplot` for each categorical column, arrange the `countplot`s in a grid 
with two rows and three columns.

### Answer

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(12, 10))
columns = train.select_dtypes("category")

for i, colname in enumerate(columns):
    sns.countplot(colname, data=train, ax=fig.axes[i])
    fig.axes[i].set_title('distribution of '+colname)
plt.tight_layout()



## Question: plotting the distribution of numerical columns

For each column for which it makes sense, display its distribution using `seaborn`:
choose the most relevant type of plot for each column and display it.

**Remark.** Note that the `Age` column contains missing values:
show the number of missing / non-missing values in this column, and show the distribution of non-missing ages in a different plot.
Also, note that there is a `notna()` method for dataframes...

### Answer


In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(16, 8))

sns.distplot(train['Age'][train['Age'].notna()], ax=fig.axes[0])
sns.countplot(train['Age'].isna(),ax=fig.axes[1]).set_xticklabels(['not NaN', 'NaN'])
sns.distplot(train['Fare'], ax=fig.axes[2])
fig.axes[0].set_title('Distribution of Age')
fig.axes[2].set_title('Distribution of Fare')

plt.show()

#count plot notna() age



## Question: plot numerical columuns according to categorical columns

Do the same as in Question 4,
but **differentiate** the plots
for passengers the passengers that survived and the ones who did not.
You can use the `hue` option,
you can use a `FacetGrid`.
Look at the `seaborn` documentation and choose the best option.

### Answer

#### Facets



In [ ]:

for colonne in train.select_dtypes("category"):

    sns.catplot(data = train.sort_values(by=col), col ="Survived", x=colonne, hue=colonne, kind="count")


#### Boxplot

In [ ]:
fig, axes = plt.subplots(nrows=6 , ncols=2, figsize=(15, 40))
for i,col in enumerate(train.select_dtypes("category")):
    sns.boxplot(y="Age", x="Survived" , hue =col, data=train, ax=fig.axes[2*i])
    sns.boxplot(y="Fare", x="Survived", hue =col ,data=train, ax=fig.axes[2*i+1]).set_yscale("log")
plt.plot()

#### Catplot

In [ ]:
fig, axes = plt.subplots(nrows=6 , ncols=2, figsize=(15, 40))
for i,col in enumerate(train.select_dtypes("category")):
    sns.violinplot(y="Age", x="Survived" , hue =col, data=train, ax=fig.axes[2*i])
    sns.violinplot(y="Fare", x="Survived", hue =col ,data=train, ax=fig.axes[2*i+1])
plt.show()




## Question: categorical plots, barplots

The `Titanic`  dataset may be compared with datasets generated by
*surveys* (*questionnaires*). In such datasets, each row correspond
to an individual. An individual is described by *demographic* variables
and *attitudinal/response* variables. Here, `Age, Sex, Pclass, Embarked, ...`
are considered as demographic variables while `Survived` is
regarded as an attitudinal/response variable. It is desirable to plot the response
variable as a function of (some of) the demographic variables.
All variables are categorical.

### Answer



In [ ]:
fig, axes = plt.subplots(nrows=6 , ncols=2, figsize=(15, 30))
fig.suptitle('Barplot or Age and Fare facet by survival')    
for i,col in enumerate(train.select_dtypes("category")):
    sns.barplot(y="Age", x="Survived" , hue =col, data=train, ax=axes[i,0])
    sns.barplot(y="Fare", x="Survived", hue =col ,data=train, ax=axes[i,1])
plt.show()


Barplots do not give us hints about the size of the different sub-populations.


## Question: categorical plots, mosaicplots



You have drawn a `boxplot` of `Age` with respect to passenger class (`Pclass`).
It would be nice to complement the plot with an indication of the number of missing values.
It would also make sense to indicate the number of individuals in each subsample.

Mosaicplot as delivered by module `statsmodels` is a very
powerful tool to visualize *contingency tables*.

A *two-way contingency table* and the associated `mosaicplot` reveal that
the passengers who boarded in the three harbors (Southampton, Cherbourg, Q)
did not buy the same kind of tickets. Half of the passengers embarked at Cherbourg
travelled first class while at most one fourth of the passengers embarked
at Southampton did.

### Answer



In [ ]:
fig, axes = plt.subplots(nrows = 2, figsize=(10, 16))

nan_ticks =["Na ="] + train[train['Age'].isna()].groupby(['Survived','Pclass'])['PassengerId'].count().to_list()
n_ticks = ["n ="] + train[train['Age'].notna()].groupby(['Survived','Pclass'])['PassengerId'].count().to_list()

    
for i, tick in enumerate(nan_ticks):
    posx=0.25*(i-2)
    if i>3: posx+=0.25
    axes[0].text(posx, -2, tick, fontsize=10, color='red')
    axes[0].text(posx, +5, n_ticks[i], fontsize=12, color = 'blue')


bxp = sns.boxplot(y="Age", x="Survived" , hue = "Pclass", data=train, ax=fig.axes[0])
fig.axes[0].set_title("boxplot of age by Pclasse and survival")

mosaic(train,['Pclass', 'Embarked'], gap=0.02, ax=fig.axes[1],title='Class by emarkment port', axes_label=True)
plt.show()


## Question: three-way mosaicplots

A three-way contingency illustrate the association between `Sex`, `Pclass`
and `Survived`.

### Answer



In [ ]:
fig, axes = plt.subplots(figsize=(12, 10))

mosaic(train,['Pclass', 'Sex', 'Survived'], gap =0.02, ax=axes, title='contingency of Pclass, Sex, Survived')
plt.show()


## Question: roommates

For each passenger, compute the number of passengers in the same cabin (including himself).
Add a column called `Roommates` that contain this information.
When the cabin information is missing, then the `Roommates` is missing as well.
You will need to use `groupby` and `join` operations.



**Warning.** You should not `reset_index` after the `groupby` operation if you `join` with the resulting dataframe.


We may proceed by projecting on columns
`PassengerId,  and `Cabin` while filtering (selecting) the rows
where `Cabin` is available. This is the `df_cabin` dataframe below.





- Plot `Roommates` according to `Pclass`
- Plot `Roommates` according to `Embarkment`
- Plot `Roommates` according to `Sex`

### Answer

In [ ]:

cab = train.groupby("Cabin", as_index=False)["PassengerId"].count()
cab.columns =["Cabin", "Roommates"]
cab.head()

if 'Roommates' not in train:
    train = train.join(cab.set_index("Cabin"), on="Cabin")


In [ ]:
train.head()



## Question: plotting  `RoomMates` column.


### Answer



In [ ]:
fig, axes = plt.subplots(nrows=3 , ncols=2, figsize=(15, 20))
cols = ['Pclass', 'Sex', 'Embarked']

plt.suptitle('Distribution of Roomates by Category')

for i,c in enumerate(cols):
    sns.countplot(data=train, x=c, hue='Roommates', ax=axes[i,0])
    sns.countplot(data=train, hue=c, x='Roommates', ax=axes[i,1])




## Question: surviving together

Is there a cabin where some passengers survived while others did not?


### Answer



In [ ]:
surv = train.groupby('Cabin')['Survived'].value_counts().unstack().fillna(0)
surv.columns =["False", "True"]

#find cabins with survivors and non surivors
surv[(surv['True']>0) & (surv['False']>0)]



## Question: Missing `Cabin`  value and `Age`


The `Cabin` column is `NaN` for many rows, how are
those missing data distributed acrross the classes?

Plot the age distribution with
respect to the availability of the `Cabin` column (`Cabin.isna()`).

### Answer


In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(12, 8))
fig.suptitle('Age distribution according to the availability of cabins')
nan_cabin = train[train['Cabin'].isna()]

sns.countplot("Pclass", data= nan_cabin, ax=fig.axes[0])
sns.distplot(nan_cabin["Age"][nan_cabin['Age'].notna()], ax=fig.axes[1])
plt.show()



## Question: Milking the `Name` column

Column `Name` gathers different kind of information pertaining
to a passenger: name, civility,
and given name(s). For some female passengers, the apparent given names
are actually the husband's name and the maidens name is indicated between
parentheses.

The information contained in the `Name` column Has to be extracted in a proper way:
The string should be into three parts: the first one containing the name,
the second one containing the civility  and, the last one
possibly containing given names or given names plus maiden names. Passing is
facilitated by the next observation: the name is followed by a comma, the
civility is followed by a period. You may use regular expressions
as providede by  module `re`  or module `regex`.


- Split the information contained in `Name` into three columns `Name`, `Given_names`, and `Civility`.
- Go one step further. Extract the *maiden name* and actual *given names* when information available.

At the end of the day, column `Name` should contain the *maiden name* for married female passengers
Column `Given_Names` should contain the real given names of the passenger, not the ones of the spouse.

To learn more about Regular Expressions see the documentation
of the [`re` module](https://docs.python.org/3/library/re.html#module-re)
and the tutorial [Howto](https://docs.python.org/3/howto/regex.html#regex-howto)

Understand the difference between a `string` and a `raw string`.



In [ ]:
print(r"\n is not a newline", "\nis a newline\n", "\\\\n is not a newline\n")


### Answer



In [ ]:
def parse_name(s):
    """Parse name field in Titanic data frame.

    ACHTUNG: there are some outliers with given names containing parenthsized items. Some people travelled under pseudonyms (see [Henry Samuel Morley](https://www.encyclopedia-titanica.org/titanic-victim/henry-samuel-morley.html)).
    """
    name = s
    civility, Spouse_name, Maiden_name, pseudonym, given_names = None, None, None, None, None
    
    # TODO:
    civility = regex.search(r'\,\s(.*?)\.', s)[0].strip(", ")
    name = regex.search(r'(.*?)\,', s)[0].strip(",")
    prth = regex.search(r'\((.*?)\)', s)
    
    pseudo =  regex.search(r'\"(.*?)\"', s)
    if pseudo:
        pseudonym = pseudo[0]
    
    if prth:
        if not regex.search(r'\(\"(.*?)\"\)', prth[0]):
            Maiden_name = prth[0].strip("()").split(" ")[-1]
            given_names = " ".join(prth[0].strip("()").split(" ")[:-1])
            Spouse_name = name
            name += " " + given_names
        else:
            given_names = regex.search(r'\.(.+?)\(', s)[0].strip(". (") + " " + pseudonym
            name += " " + given_names
        
    else:
        given_names = regex.search(r'\.(.+?)$', s)[0].strip(". (") 
        name += " " + given_names
        
        
    return {'Name': name,
            'Civility': civility,
            'Given_names': given_names,
            'Spouse_name': Spouse_name,
            'Maiden_name': Maiden_name}




## Question: about `map` objects

What is a `map object`? Is it a sequence (list, tuple, etc)? Is it
an `Iterable`? Something else? You may use function `isinstance` and
submodule `collections.abc` to settle these questions?

### Answer



In [ ]:

ex = (1, 2, 3)
def double(n):
    n=n*2
    
m = map(double, ex)
print(m)

print("Iterable ?", isinstance(m, abc.Iterable))
print("Sequence ?", isinstance(m, abc.Sequence))



## Question: aristocrats on board

Some passengers were of noble extraction, or officers in some Army.
Point them.

### Answer



In [ ]:
parsed_names = pd.DataFrame(list(train["Name"].map(parse_name)))
civ_grouped = parsed_names.groupby(['Civility'])['Name'].count()
civ_grouped

In [ ]:
#nobles
parsed_names[parsed_names["Civility"].isin(['Don.', 'Lady.', 'Sir.', 'the Countess.', 'Jonkheer.'])]

In [ ]:
#army
parsed_names[parsed_names["Civility"].isin([ 'Major.' ,'Col.', 'Capt.'])]

In [ ]:
#other titles Doctor and Reverend
parsed_names[parsed_names["Civility"].isin(['Rev.', 'Dr.'])] 


## Question: How many people are given Civility 'Mrs', 'Miss', 'Mr', 'Master'?


### Answer



In [ ]:
civ_grouped = parsed_names.groupby(['Civility'])['Name'].count()
civ_grouped[['Mrs.','Mr.','Miss.','Master.']]


ACHTUNG: there are some outliers with given names containing parenthsized items. Some people travelled under pseudonyms (see [Henry Samuel Morley](https://www.encyclopedia-titanica.org/titanic-victim/henry-samuel-morley.html)).





## Question:  For each embarkment and sex, visualize the survival ratio.


### Answer



In [ ]:

fig, ax = plt.subplots(ncols = 2, figsize=(10,7))
fig.suptitle(' survival ratio, according to embarkment and sex')
train.groupby('Survived')['Sex'].value_counts().unstack().apply(lambda x : x/sum(x)).stack().unstack('Survived').plot(ax=fig.axes[0], kind='bar', stacked=True)
train.groupby('Survived')['Embarked'].value_counts().unstack().apply(lambda x : x/sum(x)).stack().unstack('Survived').plot(ax=fig.axes[1], kind='bar', stacked=True)

In [ ]:
tt = train.groupby(['Sex','Embarked','Survived'])["PassengerId"].count()
tt = tt.unstack('Sex').unstack('Embarked')
tt=tt.apply(lambda x : x/sum(x)).stack('Embarked').unstack('Survived')

fig, ax = plt.subplots(ncols = 2, figsize=(10,7))

cols = train['Sex'].unique()

for i in range(len(cols)):
    tt[cols[i]].plot.bar(stacked=True, ax=fig.axes[i], title = cols[i])



## Question: for each `Pclass` and `Sex`, visualize the survival ratio,

### Answer



In [ ]:

fig, ax = plt.subplots(ncols = 2, figsize=(10,7))
fig.suptitle(' survival ratio, according to Pclass and sex')
train.groupby('Survived')['Sex'].value_counts().unstack().apply(lambda x : x/sum(x)).stack().unstack('Survived').plot(ax=fig.axes[0], kind='bar', stacked=True)
train.groupby('Survived')['Pclass'].value_counts().unstack().apply(lambda x : x/sum(x)).stack().unstack('Survived').plot(ax=fig.axes[1], kind='bar', stacked=True)

In [ ]:
tt = train.groupby(['Sex','Pclass','Survived'])["PassengerId"].count()
tt = tt.unstack('Sex').unstack('Pclass')
tt=tt.apply(lambda x : x/sum(x)).stack('Pclass').unstack('Survived')

fig, ax = plt.subplots(ncols = 2, figsize=(10,7))

cols = train['Sex'].unique()

for i in range(len(cols)):
    tt[cols[i]].plot.bar(stacked=True, ax=fig.axes[i], title = cols[i])



## Question: compute pairwise correlations between numerical columns.


### Answer



In [ ]:
tmp = train[['Age','Fare','Pclass', 'Roommates']][(train.isna().sum(axis=1) == 0) & (train['Fare'] >= 1)]

tmp['Fare']=tmp['Fare'].apply(np.log10)

x= sns.pairplot(tmp,  height=4)
x.fig.suptitle('correlations between numerical columns', fontsize = 16)
plt.tight_layout()



## Pandas and SQL

A systematic comparison between Pandas and SQL querying can be found
at
[https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sql.html](https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sql.html)


## Pandas pipeliness

Loading and transforming datasets is a cumbersome task. It is very likely that
the sequence of transformations you have performed on the `train.csv`  dataset will
be carried out on other datasets (just think about `test.csv`). For the sake of
reproducibility, it is convenient to build a *transformation pipeline*.
The `pipe` method for `DataFrame` class provides us with a means to do that.

Pipelining DataFrame methods is easy, it is enough to write:



In [ ]:
# df.meth1(arg11, arg12)\
#   .meth2(arg21)\
#   .meth3(arg31, arg32, arg33)\
#   ...



### `pipe` method for class `DataFrame`


The `pipe` method allows us to chain transformations implemented by functions.
Pipelining ordinary functions can be done using the `pipe` method of
class `DataFrame`.





### Module `pdpipe`

Module [`pdpipe`](https://pdpipe.github.io/pdpipe/) is more ambitious.

A *transformation pipeline* defines a sequence of operations that should be applied
to a dataframe. In the `pdpipe` framework, it is an instance of class `pdpipe.PdPipeLine`





## Question: use `pdpipe`

Build a pipeline of transformatioms
using the [pdpipe](https://pdpipe.github.io/pdpipe/doc/pdpipe/) module.

### Answer





## Miscellanea

Browse the `seaborn` documentation in order to make up your
mind on the following questions.

1. What is a *hue*?
1. Is the *colorPalette* part of style?
1. What is a saturation channel?
1. What is a saturation channel made for?
1. What is an *aesthetic parameter*?
1. What is the type of `seaborn.FacetGrid`?




In [ ]:

"""
1. we use a hue to determine which column in the dataframe should be used for colour encoding. 
It is used as a grouping variable that will produce points with different colors. It can be either categorical 
or numeric, although color mapping will behave differently in latter case.

2. There is a default value for sns.color_palette() (or sns.set_palette()), if we don't specify a set of color.
But we can also create our own color palette by choosing a list of colors in any valid Matplotlib format.

3.  All possible colors can be specified according to hue, saturation, and brightness in an RGB model, for every pixel.
Saturation is an expression for the relative bandwidth of the visible output from a light source.

4. When saturation increases, colors appear more pure, we can use the desaturate(color, prop) function of seaborn
to decrease the saturation by "prop" percent.
And when it decreases, colors appear more faded.

5. The aesthetic parameter allows us to control the style and to scale the plot of our figures.
We can set the style by setting some parameters (style, palette, font, font_scale, color_codes)

6. The type of seaborn.FacetGrid is 'seaborn.axisgrid.FacetGrid'
"""

k = sns.FacetGrid(data=train)
plt.close()
type(k)
